# Подготовка

## Установка зависимостей

In [15]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.27.post2 peft trl triton==3.1.0
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer openai-whisper soundfile librosa pydantic
!pip install --no-deps unsloth ffmpeg
!pip install flash-attn --no-build-isolation --no-cache-dir

## Сервисные функции

Функция выделения аудио из видео файла

In [16]:
import subprocess
import os

def extract_audio(video_path):
  """
    Извлекает аудиодорожку из видеофайла и сохраняет её в формате WAV.

    Параметры:
    video_path (str): Путь к исходному видеофайлу.

    Возвращает:
    str: Путь к сохранённому аудиофайлу.

    Примечание:
    - Используется `ffmpeg` для обработки видеофайла.
    - Аудио сохраняется в формате PCM 16 бит с частотой дискретизации 16 кГц.
    - Если файл уже существует, он будет перезаписан.
    """
  os.mkdir("./outputs")
  audio_path = "./outputs/extracted_audio.wav"

  subprocess.run([
      "ffmpeg", "-i", video_path, "-acodec", "pcm_s16le", "-ar", "16000", audio_path, "-y"
  ])

  print("Аудио успешно извлечено:", audio_path)
  return audio_path

Функция разбиения аудио на части

In [17]:
import librosa
import numpy as np
import soundfile as sf
import os

def split_audio(audio_path):
    """
    Разбивает аудиофайл на фрагменты продолжительностью 30 секунд и сохраняет их в отдельные файлы.

    Параметры:
    audio_path (str): Путь к исходному аудиофайлу.

    Возвращает:
    list: Список путей к сохранённым аудиофрагментам.

    Примечание:
    - Аудиофайл загружается с частотой дискретизации 16 кГц.
    - Каждый фрагмент сохраняется в формате WAV.
    - Фрагменты сохраняются в папке `./outputs/chunks`.
    - Если папка `chunks` не существует, она создаётся автоматически.
    """
    # Загружаем аудио файл
    chunks_path="./outputs/chunks"
    os.mkdir(chunks_path)
    audio, sr = librosa.load(audio_path, sr=16000)  # Проверяем формат 16kHz
    chunk_duration = 30  # Выбираем отрезки по 30 секунд
    chunk_samples = chunk_duration * sr  # Количество частей

    # Разбиваем на части
    chunks = [audio[i : i + chunk_samples] for i in range(0, len(audio), chunk_samples)]

    # Сохраняем части в отдельные WAV файлы
    chunk_paths = []
    for i, chunk in enumerate(chunks):
        chunk_file = f"{chunks_path}/audio_chunk_{i}.wav"
        sf.write(chunk_file, chunk, sr)
        chunk_paths.append(chunk_file)

    print(f"Аудио разделено на {len(chunk_paths)} частей.")

    return chunk_paths

Функция транскрибации

In [18]:
import torch
import torch.nn as nn
import gc
import shutil
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from tqdm import tqdm
from huggingface_hub import hf_hub_download

class LoRALayer(nn.Module):
    """
    Реализация LoRA (Low-Rank Adaptation) для линейных слоев.

    Параметры:
    - base_layer (nn.Module): Исходный линейный слой, к которому применяется LoRA.
    - r (int): Ранг разложения LoRA (по умолчанию 8).
    - alpha (int): Коэффициент масштабирования (по умолчанию 32).
    - dropout (float): Вероятность dropout между LoRA-матрицами (по умолчанию 0.2).

    Описание:
    - Разлагает линейный слой на две дополнительные матрицы (A и B).
    - Использует dropout между матрицами для регуляризации.
    - После обработки LoRA-слой суммируется с выходом оригинального слоя.
    """

    def __init__(self, base_layer, r=8, alpha=32, dropout=0.2):
        super().__init__()
        self.base_layer = base_layer  # Оригинальный слой
        self.r = r  # Ранк
        self.scaling = alpha / r

        # LoRA A и B матрицы
        self.lora_A = nn.Linear(base_layer.in_features, r, bias=False)
        self.lora_B = nn.Linear(r, base_layer.out_features, bias=False)

         # Dropout между A и B
        self.dropout = nn.Dropout(p=dropout)

        # Инициализируем LoRA веса
        nn.init.kaiming_uniform_(self.lora_A.weight)
        nn.init.zeros_(self.lora_B.weight)

    def forward(self, x):
        return self.base_layer(x) + self.lora_B(self.lora_A(x)) * self.scaling

def get_transcription(chunk_paths):
  """
  Функция для транскрипции аудиофайлов с использованием модели Whisper и LoRA.

  Параметры:
  - chunk_paths (list): Список путей к аудиофрагментам.

  Возвращает:
  - str: Полная транскрипция аудиофайлов.

  Описание:
  1. Определяет, доступен ли GPU и загружает модель Whisper.
  2. Загружает процессор Whisper для обработки аудио.
  3. Применяет принудительную настройку языка (русский) для декодера.
  4. Заменяет линейные слои модели на LoRA-слои.
  5. Загружает предварительно обученные LoRA-веса из Hugging Face.
  6. Переводит модель в режим инференса, отключая dropout.
  7. Использует `torch.compile()` для ускорения работы модели (если поддерживается).
  8. Обрабатывает каждую часть аудио, проводя транскрипцию с Whisper.
  9. Собирает транскрибированные части в единый текст.
  10. Освобождает память, очищая кэш CUDA и сборщик мусора.
  """

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # HuggingFace репозиторий для модели
  repo_id = "UDZH/whisper-small-lora-finetuned-ru"
  filename = "whisper_lora_weights.pth"
  lora_path = hf_hub_download(repo_id=repo_id, filename=filename)

  # Загружаем процессор и базовую модель Whisper
  model_name="openai/whisper-small"
  processor = WhisperProcessor.from_pretrained(model_name)
  whisper_model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

  # Принудительное использование русского языка для декодера
  whisper_model.config.forced_decoder_ids = processor.get_decoder_prompt_ids(language="russian", task="transcribe")

  # Отключаем dropout перед инференсом
  for name, module in whisper_model.named_modules():
      if isinstance(module, nn.Linear) and any(k in name for k in ["q_proj", "v_proj", "k_proj", "out_proj", "fc1", "fc2"]):
          parent = whisper_model.get_submodule(".".join(name.split(".")[:-1]))
          lora_layer = LoRALayer(module, r=16, alpha=32, dropout=0.4).to(device)
          setattr(parent, name.split(".")[-1], lora_layer)

  # Загружаем веса LoRA и конвертируем их в FP16
  lora_weights = torch.load(lora_path, map_location=device)
  for k, v in lora_weights.items():
      lora_weights[k] = v.half()  # Преобразуем в FP16

  # Загружаем LoRA веса в модель
  missing_keys, unexpected_keys = whisper_model.load_state_dict(lora_weights, strict=False)
  print("LoRA веса загружены.")
  print(f"Пропущенные ключи: {missing_keys}")
  print(f"Неожиданные ключи: {unexpected_keys}")

  # Переводим в режим инференса и полностью отключаем dropout
  whisper_model.eval()
  for name, module in whisper_model.named_modules():
      if isinstance(module, nn.Dropout):
          module.p = 0.0  # Полностью отключаем dropout

  # Применяем torch.compile() для ускорения (если поддерживается)
  try:
      whisper_model = torch.compile(whisper_model)
      print("torch.compile() успешно применён для ускорения инференса!")
  except AttributeError:
      print("torch.compile() не поддерживается в данной версии PyTorch, пропускаем...")

  # Проводим транскрипцию для каждой части
  transcriptions = []

  # Создаем chunk_paths как список путей до частей аудио
  for chunk_file in tqdm(chunk_paths, desc="Обработка частей", unit="chunk"):
      # Загружаем часть аудио (должно быть 16kHz для Whisper)
      audio, _ = librosa.load(chunk_file, sr=16000)

      # Токенизируем взодные данные для Whisper
      input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(device)

      # Транскрибируем
      with torch.no_grad():
          predicted_ids = whisper_model.generate(input_features)

      transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
      transcriptions.append(transcription)

  # Склеиваем результаты частей в общую транскрипцию
  full_transcription = " ".join(transcriptions)

  # Удаляем все аудио файлы
  outputs_path = "./outputs"
  shutil.rmtree(outputs_path)

  # Чистим память и зависшие в памяти тензоры
  del whisper_model
  del processor
  del transcriptions
  del transcription
  del chunk_file
  del audio
  del input_features

  for obj in gc.get_objects():
      try:
          if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
              del obj
      except:
          pass

  gc.collect()
  torch.cuda.empty_cache()
  torch.cuda.ipc_collect()

  return full_transcription

Функции обработки текста

In [19]:
import re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')

def clean_transcript(input_text):
    """
    Функция для очистки транскрипции от шумовых фраз, повторов и лишних символов.

    Параметры:
    - input_text (str): Исходная транскрипция.

    Возвращает:
    - str: Очищенный и структурированный текст.

    Описание этапов:
    1. **Удаление шумовых фраз** – исключает часто встречающиеся бессмысленные фразы и слова-паразиты.
    2. **Удаление повторов слов** – убирает последовательные дублирующиеся слова (например, "финансист финансист" → "финансист").
    3. **Сокращение длинных повторяющихся букв** – заменяет длинные последовательности одной буквы (например, "ээээ" → "э").
    4. **Разбиение на предложения** – использует `nltk.sent_tokenize()` для разбиения текста на осмысленные фрагменты.
    5. **Фильтрация коротких предложений** – удаляет слишком короткие предложения (менее 3 слов), так как они, скорее всего, являются шумом.
    6. **Удаление случайных чисел** – устраняет отдельно стоящие числа, которые могут быть артефактами транскрипции.
    7. **Сборка итогового текста** – объединяет отфильтрованные предложения обратно в структурированный текст.
    """

    # Удаляем бессмысленные фразы и шум
    common_phrases = [
    r"\bсбер какие\b", r"\bджой какие\b", r"\bафина какие\b", r"\bтихо\b",
    r"\bкакие деньги\b", r"\bкакие вопросы\b", r"\bкакие виды\b",
    r"\bкакие расклады\b", r"\bкакие правила\b", r"\bкакие кредиты\b",
    r"\bкакие платежные\b", r"\bкакие планы\b", r"\bкакие условия\b",
    r"\bкакие виды у фильма\b", r"\bкакие виды у города\b",
    r"\bкакие виды у банка\b", r"\bкакие вопросы мне нужны\b",
    r"\bкакие поечару\b", r"\bкакие как там\b", r"\bкакие у города\b",
    r"\bкакие какие\b", r"\bкакие виды говорить\b", r"\bкакие что делать\b",
    r"\bпожалуйста\b", r"\bтихо\b", r"\bафина\b", r"\bджой\b",
    r"\bменьше часов по работе\b", r"\bвопросы у моей семьи\b",
    r"\bбудем оттуда\b", r"\bбудем менять тогда\b", r"\bпродолжилась там же\b",
    r"\bсчетом объема\b", r"\bкуда же вы\b", r"\bкуда идем\b",
    r"\bгромче если можно\b", r"\bкак называется\b", r"\bэто тоже важно\b",
    r"\bработаем не работаем\b", r"\bэто понятно здесь\b", r"\bчто нам нужно\b",
    r"\bпосмотреть афина\b", r"\bпосмотреть как бы\b", r"\bпосмотреть пробуем\b",
    r"\bкуда перейдем\b", r"\bну то есть\b", r"\bто есть\b", r"\bи уже от этого\b",
    r"\bвиды у фильма\b", r"\bвиды \b",r"\bафина\b", r"\bджой\b",r"\bсбер\b",
    r"\bсалют среднее расстояние между нептуном и солнцем\b", r"\bсалют\b"]

    for phrase in common_phrases:
        input_text = re.sub(phrase, "", input_text, flags=re.IGNORECASE)

    # Убираем повторы слов (например, "финансист финансист" → "финансист")
    input_text = re.sub(r"\b(\w+)\s+\1\b", r"\1", input_text, flags=re.IGNORECASE)

    # Убираем длинные повторяющиеся буквы (например, "ээээ" → "э")
    input_text = re.sub(r"([a-zA-Zа-яА-Я])\1{3,}", r"\1", input_text)

    # Разбиваем текст на предложения
    sentences = sent_tokenize(input_text)

    # Фильтруем слишком короткие и бессмысленные предложения
    sentences = [sent for sent in sentences if len(sent.split()) > 3]

    # Убираем случайные цифры, стоящие отдельно (например, "два три 9" → "два три")
    sentences = [re.sub(r"\b\d+\b", "", sent) for sent in sentences]

    # Объединяем обратно в текст
    cleaned_text = " ".join(sentences)

    return cleaned_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Функция выделения ключевой информации из транскрипции

In [20]:
import unsloth
import torch
import gc
import json
import re
import time
import ast
from tqdm import tqdm
from pydantic import BaseModel, ValidationError
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported

# Определяем устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Определяем структуру JSON-ответа с помощью Pydantic
class MeetingSummary(BaseModel):
    Summarization: str
    Topics: list[str]
    Actions: list[str]
    Problems: list[str]
    Decisions: list[str]

# Эталонный JSON-шаблон
REFERENCE_JSON = {
    "Summarization": "Brief meeting summary...",
    "Topics": ["Topic 1", "Topic 2"],
    "Actions": ["Action 1", "Action 2"],
    "Problems": ["Problem 1", "Problem 2"],
    "Decisions": ["Decision 1", "Decision 2"]
    }

# Фиксированный промпт без транскрипта
PROMPT_TEMPLATE = """
Analyze the following meeting transcript and extract the key points:
1. **Summarization** – a brief summary of the meeting.
2. **Topics** – a list of topics discussed.
3. **Decisions** – key decisions made.
4. **Problems** – challenges or issues identified.
5. **Actions** – planned or taken actions.

Return the output **STRICTLY in the following JSON format**:
{{
  "Summarization": "Brief meeting summary...",
  "Topics": ["Topic 1", "Topic 2"],
  "Actions": ["Action 1", "Action 2"],
  "Problems": ["Problem 1", "Problem 2"],
  "Decisions": ["Decision 1", "Decision 2"]
}}

Meeting transcript (in Russian):
{transcript}

**Return only a valid JSON response in Russian language.**
**Do not include explanations, introductions, or extra text.**
**If a category is missing, return an empty array [].**

### Response:
"""

def clean_json_fields(data):
    """
    Очищает JSON, исправляя возможные ошибки со списками.
    """
    for key in ["Topics", "Actions", "Problems", "Decisions"]:
        if isinstance(data.get(key), list):
            cleaned_list = []
            for item in data[key]:
                if isinstance(item, str) and item.startswith("["):
                    try:
                        cleaned_list.extend(ast.literal_eval(item))
                    except (SyntaxError, ValueError):
                        cleaned_list.append(item)
                else:
                    cleaned_list.append(item)
            data[key] = cleaned_list
    return data

def extract_valid_json(text):
    """
    Извлекает первый JSON, который отличается от шаблонного.
    """
    text = text.strip()
    text = re.sub(r"^### Ответ:\s*", "", text)
    text = re.sub(r"^Ответ:\s*", "", text)

    json_matches = re.findall(r"\{[\s\S]*?\}", text)
    for json_text in json_matches:
        try:
            extracted_json = json.loads(json_text)
        except json.JSONDecodeError:
            try:
                extracted_json = ast.literal_eval(json_text)
            except (ValueError, SyntaxError):
                continue

        extracted_json = clean_json_fields(extracted_json)
        if extracted_json and extracted_json != REFERENCE_JSON:
            return extracted_json
    return None

def split_transcript(tokenizer, transcript, max_tokens=3000, overlap=300):
    """
    Разбивает стенограмму на части с перекрытием.
    """
    tokens = tokenizer.tokenize(transcript)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk = tokenizer.convert_tokens_to_string(tokens[start:end])
        chunks.append(chunk)
        start += max_tokens - overlap
    return chunks

def generate_response(model, tokenizer, chunk: str) -> str:
    """
    Генерирует ответ модели на основе переданного чанка стенограммы.
    """
    prompt = PROMPT_TEMPLATE.format(transcript=chunk)
    inputs = tokenizer([prompt], return_tensors="pt", truncation=True, max_length=4096).to(device)
    with torch.inference_mode():
        output_ids = model.generate(**inputs, max_new_tokens=200, do_sample=False)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

def summarize_chunk(model, tokenizer, chunk):
    """
    Обрабатывает отдельный чанк стенограммы, создавая JSON-резюме.

    Параметры:
    - model: Модель для обработки текста.
    - tokenizer: Токенизатор модели.
    - chunk (str): Часть стенограммы.

    Возвращает:
    - dict: JSON-объект с ключевыми моментами.
    """
    response = generate_response(model, tokenizer, chunk)
    json_data = extract_valid_json(response)
    return json_data if json_data else {}

def merge_chunks(chunks):
    """
    Объединяет JSON-резюме всех чанков в один итоговый JSON.

    Параметры:
    - chunks (list): Список JSON-объектов с ключевыми моментами.

    Возвращает:
    - dict: Итоговое объединённое резюме.
    """
    if not chunks:
        return {
            "Summarization": "",
            "Topics": [],
            "Actions": [],
            "Problems": [],
            "Decisions": []
        }

    return {
        "Summarization": " ".join(chunk.get("Summarization", "") for chunk in chunks),
        "Topics": list(set(topic for chunk in chunks for topic in chunk.get("Topics", []))),
        "Actions": list(set(action for chunk in chunks for action in chunk.get("Actions", []))),
        "Problems": list(set(problem for chunk in chunks for problem in chunk.get("Problems", []))),
        "Decisions": list(set(decision for chunk in chunks for decision in chunk.get("Decisions", []))),
    }

def extract_meeting_summary(model, tokenizer, input_text: str, retry_attempts=3):
    """
    Обрабатывает стенограмму и извлекает ключевые моменты встречи.

    Параметры:
    - model: Модель для обработки текста.
    - tokenizer: Токенизатор модели.
    - input_text (str): Входной текст стенограммы.
    - retry_attempts (int): Количество повторных попыток в случае неудачи.

    Возвращает:
    - dict: Итоговое резюме встречи.
    """
    for _ in range(retry_attempts):
        chunks = split_transcript(tokenizer, input_text)
        chunk_summaries = [summarize_chunk(model, tokenizer, chunk) for chunk in chunks]

        if chunk_summaries:
            return merge_chunks(chunk_summaries)

    return {
        "Summarization": "",
        "Topics": [],
        "Actions": [],
        "Problems": [],
        "Decisions": []
    }

def get_meeting_info(input_text):
    """
    Основная функция для обработки стенограммы встречи.

    Параметры:
    - input_text (str): Входной текст стенограммы.

    Возвращает:
    - dict: Итоговое резюме встречи.
    """

    model_name = "UDZH/deepseek-meeting-summary"

    deepseek_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name, max_seq_length=16_384, dtype=None, load_in_4bit=True, device_map='auto',
        trust_remote_code=True
    )
    deepseek_model = FastLanguageModel.for_inference(deepseek_model)
    result = extract_meeting_summary(deepseek_model, tokenizer, input_text)

    del deepseek_model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    return result

Форматирование вывода для чтения

In [21]:
import re

def format_transcription(text):
  """
  Форматирует текст стенограммы, разбивая его на отдельные предложения.

  Параметры:
  - text (str): Исходный текст транскрипции.

  Возвращает:
  - str: Отформатированный текст, где каждое предложение начинается с новой строки.

  Описание:
  1. Разбивает текст на предложения по знакам окончания (. ! ?), при этом знаки остаются в тексте.
  2. Использует `re.split()` с регулярным выражением для корректного разбиения.
  3. Удаляет лишние пробелы в начале и конце текста.
  4. Объединяет предложения, размещая каждое на новой строке.
  """

  # Разбиваем текст по знакам окончания предложений (. ! ?), оставляя их в тексте
  sentences = re.split(r'(?<=[.!?])\s+', text.strip())
  # Выводим каждое предложение на новой строке
  formatted_text = "\n".join(sentences)
  return formatted_text

In [22]:
import ast

def clean_list(lst):
    """
    Очищает список от вложенных строковых списков, убирает дубликаты и лишние символы.

    Параметры:
    - lst (list): Исходный список строк.

    Возвращает:
    - list: Очищенный список без дубликатов и лишних символов.

    Описание:
    1. Убирает кавычки («»), пробелы в начале и конце строк.
    2. Проверяет, содержится ли элемент в формате строкового списка (например, '["a", "b"]') и преобразует его в список.
    3. Разбивает элементы, содержащие запятые, и добавляет их в список отдельно.
    4. Удаляет дубликаты, сохраняя только уникальные элементы.
    """

    seen = set()
    cleaned_list = []

    for item in lst:
        if isinstance(item, str):
            item = item.strip().replace("«", "").replace("»", "")

            # Проверяем, является ли item строковым списком (например, '["a", "b"]')
            if item.startswith("[") and item.endswith("]"):
                try:
                    parsed_item = ast.literal_eval(item)  # Преобразуем строку в список
                    if isinstance(parsed_item, list):
                        for sub_item in parsed_item:
                            sub_item = str(sub_item).strip().replace("[", "").replace("]", "")
                            if sub_item and sub_item not in seen:
                                cleaned_list.append(sub_item)
                                seen.add(sub_item)
                        continue  # Пропускаем добавление исходной строки
                except (ValueError, SyntaxError):
                    pass  # Если ошибка, рассматриваем как обычную строку

        # Разбиваем по запятым, если в строке несколько пунктов
        if "," in item:
            sub_items = [x.strip() for x in item.split(",") if x.strip()]
            for sub_item in sub_items:
                if sub_item and sub_item not in seen:
                    cleaned_list.append(sub_item)
                    seen.add(sub_item)
            continue  # Не добавляем исходную строку повторно

        if item and item not in seen:
            cleaned_list.append(item)
            seen.add(item)

    return cleaned_list

def remove_duplicates(lst):
    """
    Удаляет дубликаты из списка, сохраняя порядок элементов.

    Параметры:
    - lst (list): Исходный список строк.

    Возвращает:
    - list: Список без дубликатов, с сохранением исходного порядка.

    Описание:
    1. Приводит строки к нижнему регистру и убирает пробелы перед добавлением в `set`, чтобы избежать повторений.
    2. Сохраняет формат исходных строк (например, регистр и пробелы) при возврате результата.
    """

    seen = set()
    cleaned = []
    for x in lst:
        cleaned_x = x.strip().lower()  # Приводим к нижнему регистру, убираем пробелы
        if cleaned_x not in seen:
            seen.add(cleaned_x)
            cleaned.append(x.strip())  # Сохраняем оригинальный формат строки
    return cleaned

def clean_meeting_summary(meeting_info):
    """
    Очищает JSON-данные о встрече от дубликатов и вложенных списков.

    Параметры:
    - meeting_info (dict): JSON-объект с данными о встрече.

    Возвращает:
    - dict: Очищенные JSON-данные.

    Описание:
    1. Очищает поля `Topics`, `Actions`, `Problems` и `Decisions`, убирая дубликаты и вложенные строки-списки.
    2. Разбивает резюме встречи (`Summarization`) на предложения и убирает дубликаты.
    3. Собирает обновленные данные обратно в JSON-объект.
    """

    # Очистка списков + удаление дубликатов
    for key in ["Topics", "Actions", "Problems", "Decisions"]:
        meeting_info[key] = remove_duplicates(clean_list(meeting_info.get(key, [])))

    # Очистка резюме (по предложениям) с учётом пробелов и регистра
    if "Summarization" in meeting_info:
        sentences = [s.strip() for s in meeting_info["Summarization"].split('.') if s.strip()]
        meeting_info["Summarization"] = '. '.join(remove_duplicates(sentences))

    return meeting_info

def format_meeting_info(meeting_info):
    """
    Форматирует JSON-данные о встрече в удобочитаемый текст.

    Параметры:
    - meeting_info (dict): JSON-объект с данными о встрече.

    Возвращает:
    - str: Отформатированный текст с разделами по ключевым аспектам встречи.

    Описание:
    1. Разбивает резюме (`Summarization`) на строки для лучшей читаемости.
    2. Формирует структурированный текст с заголовками:
      - **Резюме встречи**
      - **Темы обсуждения**
      - **Действия**
      - **Проблемы**
      - **Решения**
    3. Если раздел пуст, добавляет соответствующую подпись ("Нет тем.", "Нет действий." и т. д.).
    """

    summary_text = '\n'.join(meeting_info.get('Summarization', '').split('. '))
    formatted_text = f"**Резюме встречи:**\n{summary_text}\n\n"

    formatted_text += "**Темы обсуждения:**\n"
    topics = meeting_info.get('Topics', [])
    formatted_text += "\n".join(f"- {topic}" for topic in topics) + "\n\n" if topics else "Нет тем.\n\n"

    formatted_text += "**Действия:**\n"
    actions = meeting_info.get('Actions', [])
    formatted_text += "\n".join(f"- {action}" for action in actions) + "\n\n" if actions else "Нет действий.\n\n"

    formatted_text += "**Проблемы:**\n"
    problems = meeting_info.get('Problems', [])
    formatted_text += "\n".join(f"- {problem}" for problem in problems) + "\n\n" if problems else "Нет проблем.\n\n"

    formatted_text += "**Решения:**\n"
    decisions = meeting_info.get('Decisions', [])
    formatted_text += "\n".join(f"- {decision}" for decision in decisions) + "\n" if decisions else "Нет решений.\n"

    return formatted_text

# Получение транскрипции из видео и определение ключевой информации

In [23]:
# Путь к видеофайлу с записью встречи
video_path = "./examples/meeting_video.mp4"

# Извлекаем аудиодорожку из видео
audio_path = extract_audio(video_path)

# Разбиваем аудиофайл на фрагменты
chunk_paths = split_audio(audio_path)

# Проводим транскрипцию аудиофрагментов с помощью модели распознавания речи
full_transcription = get_transcription(chunk_paths)

# Анализируем стенограмму встречи и извлекаем ключевые моменты
meeting_info = get_meeting_info(full_transcription)

# Очищаем JSON-данные встречи от дубликатов и вложенных элементов
cleaned_meeting_info = clean_meeting_summary(meeting_info)

# Преобразуем структурированные данные в удобочитаемый текстовый формат
formatted_text = format_meeting_info(cleaned_meeting_info)

# Выводим результат
print("\n-----------------------------------------------------------------")
print("Транскрипция встречи")
print("-----------------------------------------------------------------")
print(format_transcription(full_transcription))  # Форматируем стенограмму, чтобы каждое предложение было на новой строке
print("-----------------------------------------------------------------")
print("Ключевые моменты встречи")
print("-----------------------------------------------------------------")
print(formatted_text)  # Выводим структурированное резюме встречи

Аудио успешно извлечено: ./outputs/extracted_audio.wav
Аудио разделено на 97 частей.
LoRA веса загружены.
Пропущенные ключи: ['model.encoder.conv1.weight', 'model.encoder.conv1.bias', 'model.encoder.conv2.weight', 'model.encoder.conv2.bias', 'model.encoder.embed_positions.weight', 'model.encoder.layers.0.self_attn.k_proj.base_layer.weight', 'model.encoder.layers.0.self_attn.v_proj.base_layer.weight', 'model.encoder.layers.0.self_attn.v_proj.base_layer.bias', 'model.encoder.layers.0.self_attn.q_proj.base_layer.weight', 'model.encoder.layers.0.self_attn.q_proj.base_layer.bias', 'model.encoder.layers.0.self_attn.out_proj.base_layer.weight', 'model.encoder.layers.0.self_attn.out_proj.base_layer.bias', 'model.encoder.layers.0.self_attn_layer_norm.weight', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.0.fc1.base_layer.weight', 'model.encoder.layers.0.fc1.base_layer.bias', 'model.encoder.layers.0.fc2.base_layer.weight', 'model.encoder.layers.0.fc2.base_layer.bias',

Обработка частей: 100%|██████████| 97/97 [03:15<00:00,  2.02s/chunk]


Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.3.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!

-----------------------------------------------------------------
Транскрипция встречи
-----------------------------------------------------------------
you  you  you  Yn ymwneud yma, yw'r cyffredin yn ymwneud yma.
У меня слышно, Наталья?
Да, слышно.
Говорить сейчас есть возможность?
Мы говорили там...
Да, они с собой занимаются.
Поэтому есть спокойного там планения, неопременница здесь поэтому никто не нужен.
Ну понятно.
Так, чего у нас п